In [1]:
import numpy as np
from matplotlib import pyplot as plt

from qepy.calculator import QEpyCalculator 
from qepy.io import QEInput
from qepy.driver import Driver

from ase.build import bulk
from ase.lattice.spacegroup import Spacegroup
from ase.lattice.spacegroup import crystal

import pickle

/home/valeria/anaconda3/lib/python3.9/site-packages/ase/lattice/spacegroup.py:5: UserWarning: Moved to ase.spacegroup
  warnings.warn('Moved to ase.spacegroup')


In [2]:
from scipy.optimize import minimize
try:
    from mpi4py import MPI
    comm=MPI.COMM_WORLD
except:
    comm=None
path_pp='/home/valeria/Desktop/programs/dftpy/examples/ofpp/EAC/upf/blps/'
file1='si.lda.upf'
PP_list = {'Si': path_pp+file1}

In [4]:
qe_options = {
        '&control': {
            'calculation': "'scf'",
            'prefix': "'si'",
            'pseudo_dir': "'/home/valeria/Desktop/programs/dftpy/examples/ofpp/EAC/upf/blps/'",
            'restart_mode': "'from_scratch'"},
        '&system': {
            'ibrav' : 1,
            'nat': 2,
            'degauss': 0.005,
            'ecutwfc': 60,
            'occupations': "'smearing'"},
        'atomic_species': ['Si  28.08 si.lda.upf'],
        'cell_parameters angstrom': ['3.09    0.0000000000000000    0.0000000000000003',
                                     '0.0000000000000009    3.09    0.0000000000000003',
                                     '0.0000000000000000    0.0000000000000000    3.09'],
         'k_points automatic': ['20 20 20 1 1 1']}

# KS DFT
l = np.linspace(0.8, 1.4, 30)
# RHO = []
ks_ke = []
for i in np.arange(0, len(l), 1):
        n = int(i)
        X = l[n]
        si = bulk('Si', 'bcc', a= 3.09, cubic=True)
        cell = si.get_cell()
        si.set_cell(cell * X, scale_atoms=True)
        qe_options = QEInput.update_atoms(si, qe_options = qe_options,  extrapolation=False)
        QEInput().write_qe_input("/home/valeria/Documents/DFTPY/Fitting_densities/Si-bcc/DATA2/Si"+str(n)+".in", qe_options=qe_options)
        driver = Driver('/home/valeria/Documents/DFTPY/Fitting_densities/Si-bcc/DATA2/Si'+str(n)+'.in', comm=comm, logfile='/home/valeria/Documents/DFTPY/Fitting_densities/Si-bcc/DATA2/Si'+str(n)+'.out')
        driver.scf()
        driver.calc_energy()
        D = driver.get_output()
        k = [match for match in D if "Kinetic energy" in match]
        KS_KE = str(k).split()[6]
        v_men = driver.get_effective_potential().mean()
        KS_PE = driver.data2field(driver.get_effective_potential() - v_men)
        rho = driver.data2field(driver.get_density())
        ions = driver.get_dftpy_ions()
        rho.write('/home/valeria/Documents/DFTPY/Fitting_densities/Si-bcc/DATA2/rho0'+str(i)+'.xsf', ions=ions)
        driver.stop()
#         RHO.append(rho)
        ks_ke.append(KS_KE)

In [5]:
np.save("/home/valeria/Documents/DFTPY/Fitting_densities/Si-bcc/DATA2/ks_ke.npy", np.asarray(ks_ke))